In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import ConcatDataset, DataLoader
import random
import os

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from tqdm import tqdm
import copy
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler

device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

In [3]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

In [4]:
#mean and std of cifar100 dataset
mean = (0.5070751592371323, 0.48654887331495095, 0.4409178433670343)
std = (0.2673342858792401, 0.2564384629170883, 0.27615047132568404)
batch_size = 128
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
cifar100_training = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
cifar100_training_loader = DataLoader(cifar100_training, shuffle=True, num_workers=1, batch_size=batch_size)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
cifar100_test = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
cifar100_test_loader = DataLoader(cifar100_test, shuffle=False, num_workers=1, batch_size=batch_size)

100%|████████████████████████████████████████████████████████████████████████████████| 169001437/169001437 [00:28<00:00, 5857391.94it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


## Preparation

### Randomly select 1% of train samples

In [5]:
n_train = len(cifar100_training)
idx_train = np.random.permutation(n_train)
idx_overlap = idx_train[:int(n_train * 0.01)]

### Split the overlap set into 20 / 80

In [8]:
# Calculate the lengths of the validation and test subsets
val_len = int(len(idx_overlap) * 0.20)
test_len = len(idx_overlap) - val_len

# Split the idx_overlap into two parts with a 20/80 ratio
val_idx, test_idx = np.array_split(idx_overlap, [val_len])

In [11]:
total_idx = np.arange(n_train)
idx_nonoverlap = np.setdiff1d(total_idx, idx_overlap)
print(total_idx, len(total_idx))
print(idx_overlap, len(idx_overlap))
print(idx_nonoverlap, len(idx_nonoverlap))
print(val_idx, len(val_idx))
print(test_idx, len(test_idx))

[    0     1     2 ... 49997 49998 49999] 50000
[11841 19602 45519 25747 42642 31902 30346 12363 32490 26128 14227 26376
 44173 12968 32104 17844 43460  8369 15055  6338 15301 46250 45580 24647
 46712  4150 42460 29079 19412 34839 34478 16956 29342 26351   271 24189
  1399  5140 12649 19883 36238 34129 39915 39723  7346  8346 38676 21388
  2838 48141  1984 22320  1083 30641 27739  4622 30351 40136 28428 45494
  7187 10204 26750 44715 42944 21015 33753   913 45359 33325 16670  7496
 49735  2123 39717 12167 32266 28900 16788 16537 37875 10632 12186 32712
  2329  4937 46058 46458  9406 23165 42025 28518 43515 15009 24457  7229
 33733 10998 13443 43682 30014 41806 31462 27468 37803 45114 25041 29179
 47356  4006 38324 47427 46960 48059 39118 41256 28556 20056 21100 30695
 30995  3336  5665 28316 12359 44501 28716 27316 41791 38553 32942  6101
 36811 22326 46474 29590 43079  5944  7107 20114 29473 30098 49646 22468
 44302 13919 31275 14991 36373  4130  5093 16406 49529 37287  4775 47529
 12

In [12]:
# Define a Subset class to extract overlapped samples
class Subset(torch.utils.data.Subset):
    def __init__(self, dataset, indices):
        super().__init__(dataset, indices)
    
    def __getitem__(self, idx):
        x, y = super().__getitem__(idx)
        return x, y
    
    def get_indices(self):
        return self.indices

In [13]:
eval_val_set = Subset(cifar100_training, val_idx)
eval_test_set = Subset(cifar100_training, test_idx)
nonoverlap_set = Subset(cifar100_training, idx_nonoverlap)
new_test_set = ConcatDataset([cifar100_test, eval_val_set, eval_test_set]) 

# Print the shapes of the new test set and the overlap set
print('Val set size: ', len(eval_val_set))
print('Test set size: ', len(eval_test_set))
print('New Test set size: ', len(new_test_set))

Val set size:  100
Test set size:  400
New Test set size:  10500


In [9]:
eval_val_loader = DataLoader(eval_val_set, batch_size=1, shuffle=False)
eval_test_loader = DataLoader(eval_test_set, batch_size=1, shuffle=False)
nonoverlap_train_loader = DataLoader(nonoverlap_set, batch_size=batch_size, shuffle=True)
new_test_loader = DataLoader(new_test_set, batch_size=batch_size, shuffle=True)

### Define model structures

In [10]:
"""resnet in pytorch
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun.
    Deep Residual Learning for Image Recognition
    https://arxiv.org/abs/1512.03385v1
"""

import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    """Basic Block for resnet 18 and resnet 34
    """

    #BasicBlock and BottleNeck block
    #have different output size
    #we use class attribute expansion
    #to distinct
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        #residual function
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion)
        )

        #shortcut
        self.shortcut = nn.Sequential()

        #the shortcut output dimension is not the same with residual function
        #use 1*1 convolution to match the dimension
        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

class BottleNeck(nn.Module):
    """Residual block for resnet over 50 layers
    """
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, stride=stride, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BottleNeck.expansion, stride=stride, kernel_size=1, bias=False),
                nn.BatchNorm2d(out_channels * BottleNeck.expansion)
            )

    def forward(self, x):
        return nn.ReLU(inplace=True)(self.residual_function(x) + self.shortcut(x))

class ResNet(nn.Module):

    def __init__(self, block, num_block, num_classes=100):
        super().__init__()

        self.in_channels = 64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        #we use a different inputsize than the original paper
        #so conv2_x's stride is 1
        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        """make resnet layers(by layer i didnt mean this 'layer' was the
        same as a neuron netowork layer, ex. conv layer), one layer may
        contain more than one residual block
        Args:
            block: block type, basic block or bottle neck block
            out_channels: output depth channel number of this layer
            num_blocks: how many blocks per layer
            stride: the stride of the first block of this layer
        Return:
            return a resnet layer
        """

        # we have num_block blocks per layer, the first block
        # could be 1 or 2, other blocks would always be 1
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        output = self.conv3_x(output)
        output = self.conv4_x(output)
        output = self.conv5_x(output)
        output = self.avg_pool(output)
        output = output.view(output.size(0), -1)
        output = self.fc(output)

        return output

def resnet18():
    """ return a ResNet 18 object
    """
    return ResNet(BasicBlock, [2, 2, 2, 2])

def resnet34():
    """ return a ResNet 34 object
    """
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    """ return a ResNet 50 object
    """
    return ResNet(BottleNeck, [3, 4, 6, 3])

def resnet101():
    """ return a ResNet 101 object
    """
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    """ return a ResNet 152 object
    """
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [11]:
class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """
    def __init__(self, optimizer, total_iters, last_epoch=-1):

        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

In [12]:
def test(model: nn.Module, data_loader: torch.utils.data.DataLoader):
    model.eval()
    correct = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = F.cross_entropy(output, target)
        correct += (F.softmax(output, dim=1).max(dim=1)[1] == target).data.sum()
    return correct / len(data_loader.dataset)

In [13]:
def normal_train(model: nn.Module, optimizer: torch.optim, data_loader: torch.utils.data.DataLoader, epoch, warmup_scheduler):
    model.train()
    normal_train_loss = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        normal_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if epoch <= 1:
            warmup_scheduler.step()
    return normal_train_loss / len(data_loader)

In [14]:
def unlearn(model: nn.Module, optimizer: torch.optim, data: list):
    model.train()
    ewc_train_loss = 0
    data, target = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = -F.cross_entropy(output, target)
    ewc_train_loss += loss.item()
    loss.backward()
    optimizer.step()
    return ewc_train_loss / len(data)

In [15]:
EPOCH = 200
MILESTONES = [60, 120, 160]
SAVE_EPOCH = 10
model = resnet18().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
train_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=MILESTONES, gamma=0.2) #learning rate decay
iter_per_epoch = len(cifar100_training_loader)
warmup_scheduler = WarmUpLR(optimizer, iter_per_epoch * 1)

## Pretrain the model to get \hat\theta

In [ ]:
# Train the model
best_acc = 0.0

for epoch in range(1, EPOCH+1):
    if epoch > 1:
        train_scheduler.step(epoch)
            
    train_loss = normal_train(model, optimizer, cifar100_training_loader, epoch, warmup_scheduler)
    acc = test(model, cifar100_test_loader)
    
    print('Epoch [{}/{}], Train Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch, EPOCH+1, train_loss, 100*acc))
    
    ## start to save best performance model after learning rate decay to 0.01
    if epoch > MILESTONES[1] and best_acc < acc:
        weights_path = './ResNet18-CIFAR100/noaug-nol2/resnet18-' + str(epoch) + '.pt'
        print('saving weights file to {}'.format(weights_path))
        torch.save(model.state_dict(), weights_path)
        best_acc = acc
        continue

    if not epoch % SAVE_EPOCH:
        weights_path = './ResNet18-CIFAR100/noaug-nol2/resnet18-' + str(epoch) + '.pt'
        print('saving weights file to {}'.format(weights_path))
        torch.save(model.state_dict(), weights_path)

Epoch [1/201], Train Loss: 3.7309, Test Accuracy: 18.42%


/home/myeongseob/anaconda3/envs/msPro-dv/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:163: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [2/201], Train Loss: 2.6993, Test Accuracy: 34.84%
Epoch [3/201], Train Loss: 2.0510, Test Accuracy: 40.77%
Epoch [4/201], Train Loss: 1.6465, Test Accuracy: 46.59%
Epoch [5/201], Train Loss: 1.3078, Test Accuracy: 50.86%
Epoch [6/201], Train Loss: 1.0058, Test Accuracy: 50.18%
Epoch [7/201], Train Loss: 0.7218, Test Accuracy: 51.80%
Epoch [8/201], Train Loss: 0.4800, Test Accuracy: 52.19%
Epoch [9/201], Train Loss: 0.2972, Test Accuracy: 51.48%
Epoch [10/201], Train Loss: 0.1969, Test Accuracy: 52.86%
saving weights file to ./ResNet18-CIFAR100/noaug-nol2/resnet18-10.pt
Epoch [11/201], Train Loss: 0.1405, Test Accuracy: 53.86%
Epoch [12/201], Train Loss: 0.0721, Test Accuracy: 55.66%
Epoch [13/201], Train Loss: 0.0413, Test Accuracy: 57.07%
Epoch [14/201], Train Loss: 0.0199, Test Accuracy: 57.78%
Epoch [15/201], Train Loss: 0.0099, Test Accuracy: 58.83%
Epoch [16/201], Train Loss: 0.0049, Test Accuracy: 59.45%
Epoch [17/201], Train Loss: 0.0025, Test Accuracy: 59.34%
Epoch [18/2

### Calculate the loss for the overlapped samples - sanity check

In [ ]:
# model.load_state_dict(torch.load('./ResNet18-CIFAR100/noaug-nol2/resnet18-200.pt', map_location=torch.device('cpu'))) # checkpoint
model = model.eval()

In [ ]:
total_loss = 0.0
for data in eval_val_loader:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    total_loss += loss.item() * len(inputs)
print('Total loss on val set: %.3f' % (total_loss / len(eval_val_loader.dataset)))

In [ ]:
total_loss = 0.0
for data in eval_test_loader:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    total_loss += loss.item() * len(inputs)
print('Total loss on test set: %.3f' % (total_loss / len(eval_test_loader.dataset)))

In [ ]:
total_loss_test = 0.0
for data in nonoverlap_train_loader:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    total_loss_test += loss.item() * len(inputs)
print('Total loss on nonoverlap train set: %.3f' % (total_loss_test / len(nonoverlap_train_loader.dataset)))

In [ ]:
test(model, cifar100_test_loader)

## Hyperparameter selection based on the validation set

In [ ]:
def gen_points_on_sphere(current_point, points_count, sphere_radius):
    points_shape = (points_count,) + current_point.shape
    perturbation_direction = torch.randn(*points_shape).to(device)
    dims = tuple([i for i in range(1, len(points_shape))])
    perturbation_direction = (sphere_radius/ torch.sqrt(torch.sum(perturbation_direction ** 2, axis = dims, keepdims = True))) * perturbation_direction
    sphere_points = current_point + perturbation_direction
    
    return sphere_points.squeeze(0), perturbation_direction

In [ ]:
criterion_individual = nn.CrossEntropyLoss(reduction='none')  # Set the reduction to 'none' to compute individual losses

In [ ]:
lr_lst = [5e-3] ### you can enumerate possible parameter choice here
epoch_lst = [2]

In [ ]:
# Unlearn the model
best_lr = 0
best_epoch = 0  
best_success_top1 = 0.

for current_lr in lr_lst:
    for current_epoch in epoch_lst:
        success_top1 = 0.
        success_top5 = 0. 
        sphere_radius = 3.0

        for idx, data in enumerate(eval_val_loader):

            ### Unlearning
            model_copy = copy.deepcopy(model)
            model_copy = model_copy.train()
            
            cat_data = [torch.cat([data[0], data[0]]), torch.cat([data[1], data[1]])]
            optimizer = optim.Adam(model_copy.parameters(), lr=current_lr)    
            
            for epoch in range(1, current_epoch):
                train_loss = unlearn(model_copy, optimizer, cat_data)
                acc = test(model_copy, new_test_loader)
                print('Epoch [{}/{}], Train Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch, current_epoch, train_loss, acc*100.))

                if torch.abs(torch.tensor(train_loss)).item() < 5*1e-3:
                    break

            if torch.abs(torch.tensor(train_loss)).item() < 5*1e-3:
                trial = 0
                while True:
                    trial += 1
                    input_one_generated, _ = gen_points_on_sphere(data[0].to(device), 1, sphere_radius) ## generating the new point
                    cat_data = [torch.cat([input_one_generated, input_one_generated]), torch.cat([data[1], data[1]])]                    
                    print(f"current radius is =========================== {sphere_radius} ===========================")
                    model_copy = copy.deepcopy(model)
                    optimizer = optim.Adam(model_copy.parameters(), lr=current_lr)

                    for epoch in range(1, current_epoch):
                        train_loss = unlearn(model_copy, optimizer, cat_data)       
                        acc = test(model_copy, new_test_loader)
                        print('Generating uniform sample [%d] loss: %.3f' % (epoch, train_loss))  

                        if torch.abs(torch.tensor(train_loss)).item() < 5*1e-3:
                            break

                    if torch.abs(torch.tensor(train_loss)).item() < 1e-2:
                        sphere_radius *= 1.1  # increase sphere radius
                    else:
                        print(f'terminate the while loop : {trial}')
                        sphere_radius = 3.0
                        break
                        
            ###############################################################################################################3            
            ### score calculation corresponding to each test sample to be unlearned  
            model_copy = model_copy.eval()
            model = model.eval()

            eval_loss_lst_unlearned = []

            eval_trainloader =  DataLoader(cifar100_training, shuffle=False, num_workers=1, batch_size=512)
            for data_eval in eval_trainloader:
                inputs, labels = data_eval
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model_copy(inputs) ## difference    
                loss = criterion_individual(outputs, labels)

                outputs_orgin = model(inputs) ## difference 
                loss_origin = criterion_individual(outputs_orgin, labels)    
                eval_loss_lst_unlearned.append(np.abs((loss-loss_origin).detach().cpu().numpy()))    

            val, ind = torch.sort(torch.from_numpy(np.concatenate(eval_loss_lst_unlearned)), descending=True)

            if val_idx[idx] in [ind[0].tolist()]:
                success_top5 += 1
                success_top1 += 1      
            elif val_idx[idx] in ind[:5].tolist():
                success_top5 += 1

            print(f'Current data idx is {idx+1}')    

            print('Top1 success rate on overlap data : %.3f' % ( success_top1 / (idx+1) ))    
            print('Top5 success rate on overlap data : %.3f' % ( success_top5 / (idx+1) ))    
                
        print('Total top1 success rate on overlap data : %.3f' % (success_top1 / (len(eval_val_loader.dataset)) ))    
        print('Total top5 success rate on overlap data : %.3f' % (success_top5 / (len(eval_val_loader.dataset)) ))  
        print(f'Best lr is {best_lr}')
        print(f'Best epoch is {best_epoch}')
        
        if success_top1 / (len(eval_val_loader.dataset)) > best_success_top1:
            best_lr = current_lr
            best_epoch = current_epoch
            best_success_top1 = success_top1 / (len(eval_val_loader.dataset))

In [ ]:
print('best top1 ', best_success_top1)
print('best best_epoch ', best_epoch)
print('best best_lr ', best_lr)

## Algorithm begins to get \hat\theta_K

In [ ]:
criterion_individual = nn.CrossEntropyLoss(reduction='none')  # Set the reduction to 'none' to compute individual losses

In [ ]:
success_top1 = 0
success_top5 = 0 
sphere_radius = 3.0
for idx, data in enumerate(eval_test_loader):

    model_copy = copy.deepcopy(model)
    model_copy = model_copy.train()

    cat_data = [torch.cat([data[0], data[0]]), torch.cat([data[1], data[1]])]
    optimizer = optim.Adam(model_copy.parameters(), lr=best_lr)    

    ###############################################################################################################3            
    ## finetuning with gradient ascents 
    for epoch in range(1, best_epoch):
        train_loss = unlearn(model_copy, optimizer, cat_data)
        acc = test(model_copy, new_test_loader)
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch, best_epoch, train_loss, acc*100.))

        if torch.abs(torch.tensor(train_loss)).item() < 5*1e-3:
            break

    if torch.abs(torch.tensor(train_loss)).item() < 5*1e-3:
        trial = 0
        while True:
            trial += 1
            input_one_generated, _ = gen_points_on_sphere(data[0].to(device), 1, sphere_radius) ## generating the new point
            cat_data = [torch.cat([input_one_generated, input_one_generated]), torch.cat([data[1], data[1]])]                    
            print(f"current radius is =========================== {sphere_radius} ===========================")
            model_copy = copy.deepcopy(model)
            model_copy = model_copy.train()
            optimizer = optim.Adam(model_copy.parameters(), lr=best_lr)

            for epoch in range(1, best_epoch):
                train_loss = unlearn(model_copy, optimizer, cat_data)       
                acc = test(model_copy, new_test_loader)
                print('Generating uniform sample [%d] loss: %.3f' % (epoch, train_loss))  

                if torch.abs(torch.tensor(train_loss)).item() < 5*1e-3:
                    break

            if torch.abs(torch.tensor(train_loss)).item() < 1e-2:
                sphere_radius *= 1.1  # increase sphere radius
            else:
                print(f'terminate the while loop : {trial}')
                sphere_radius = 3.0
                break

    ###############################################################################################################3            
    ### score calculation based on the loss difference  
    model_copy = model_copy.eval()
    model = model.eval()

    eval_loss_lst_unlearned = []

    eval_trainloader =  DataLoader(cifar100_training, shuffle=False, num_workers=1, batch_size=512)
    for data_eval in eval_trainloader:
        inputs, labels = data_eval
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_copy(inputs)   
        loss = criterion_individual(outputs, labels)

        outputs_orgin = model(inputs) 
        loss_origin = criterion_individual(outputs_orgin, labels)    
        eval_loss_lst_unlearned.append(np.abs((loss-loss_origin).detach().cpu().numpy()))    

    val, ind = torch.sort(torch.from_numpy(np.concatenate(eval_loss_lst_unlearned)), descending=True)

    if test_idx[idx] in [ind[0].tolist()]:
        success_top5 += 1
        success_top1 += 1      
    elif test_idx[idx] in ind[:5].tolist():
        success_top5 += 1

    print(f'Current data idx is {idx+1}')    

    print('Top1 success rate on overlap data : %.3f' % ( success_top1 / (idx+1) ))    
    print('Top5 success rate on overlap data : %.3f' % ( success_top5 / (idx+1) ))    

print('Total top1 success rate on overlap data : %.3f' % (success_top1 / (len(eval_test_loader.dataset)) ))    
print('Total top5 success rate on overlap data : %.3f' % (success_top5 / (len(eval_test_loader.dataset)) ))  